In [21]:
import json
import sqlite3

# Load JSON data
with open("../data/faq_en.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Connect to SQLite database
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS faq (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    section_name TEXT,
    section_id TEXT,
    faq_subsection_name TEXT,
    questions TEXT,
    response TEXT,
    embeddings BLOB
)
""")

# Parse and insert data
for section in data["data"]:
    section_name = section["attributes"]["Sekcia"]
    section_id = section["attributes"]["SekciaID"]

    for subsection in section["attributes"].get("Podsekcia_s_otazkami_a_odpovedami", []):
        faq_subsection_name = subsection["Podsekcia"]
        
        for qa in subsection["Otazky_a_odpovede"]:
            question = json.dumps([qa["Otazka"]], ensure_ascii=False)
            response = qa["Odpoved"]

            cursor.execute("""
                INSERT INTO faq (section_name, section_id, faq_subsection_name, questions, response)
                VALUES (?, ?, ?, ?, ?)
            """, (section_name, section_id, faq_subsection_name, question, response))

# Commit and close connection
conn.commit()
conn.close()

print("Data inserted successfully!")


Data inserted successfully!


In [25]:
import sqlite3
import json
import litellm
import os
import numpy as np
# Set up LiteLLM with Azure OpenAI credentials
litellm.api_key = os.getenv("AZURE_OPENAI_API_KEY","6dc5202e99814e13bf43b6d9434bbaee")
litellm.api_base = os.getenv("AZURE_OPENAI_API_BASE","https://voicecast-gpt-sweden-central.openai.azure.com/")


def generate_embeddings(query):
    response = litellm.embedding(
        model="azure/text-embedding-3-large",
        deployment_id="text-embedding-3-large",
        api_version="2023-05-15",
        input=query)
    return np.array(response["data"][0]["embedding"], dtype='float32')

# Connect to SQLite
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()
# Retrieve all rows
cursor.execute("SELECT id, questions, response FROM faq")  # Adjust table name if needed
rows = cursor.fetchall()

for row in rows:
    entry_id, questions_json, response_text = row

    # Convert JSON string to list
    questions = json.loads(questions_json)

    # Combine questions and response
    combined_text = " ".join(questions) + " " + response_text

    # Get embedding
    embedding_vector = generate_embeddings(combined_text)
    embedding_blob = embedding_vector.tobytes()
    # Store in the database
    cursor.execute(
        "UPDATE faq SET embeddings = ? WHERE id = ?",
        (embedding_blob, entry_id)
    )

# Commit and close
conn.commit()
conn.close()


In [18]:
import ast
def generate_questions(question,response, n=5):
    prompt = f"""
    Your task is to generate {n} questions based on the following text:
    Text: "{response}"
    Keep in mind, that the questions should be relevant to the following example question:
    {question}
    Format your questions as a python list, each question should be in double quotes and separated by a comma.
    Example:
    [
        "Where can I donate blood?",
        "Where are the blood donation centres?",
        "Where to go for blood donation?"
    ]
    The questions should be generated based on the context of the provided text. The questions should be open-ended. 
    The questions should be in the language of the provided text.
    """
    response = litellm.completion(
        model="azure/gpt-4",
        deployment_id="gpt-4-turbo",
        messages=[{ "content": prompt,"role": "user"}])
    generated_questions = response.choices[0].message.content
    print(generated_questions)
    return generated_questions


In [2]:
import json
import sqlite3

# Load JSON data

# Connect to SQLite database
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
ALTER TABLE feedbacks ADD COLUMN TIMESTAMP float;
""")
conn.commit()
conn.close()

In [24]:

# Connect to SQLite
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()
# Retrieve all rows
cursor.execute("SELECT id, questions, response FROM faq")  # Adjust table name if needed
rows = cursor.fetchall()

for row in rows:
    entry_id, questions_json, response_text = row

    # Convert JSON string to list
    base_question = json.loads(questions_json)[0]
    if entry_id > 36:
        # Generate questions
        generated_questions = json.loads(generate_questions(base_question,response_text)) 
        generated_questions = [question.replace("\n","") for question in generated_questions]
        generated_questions.append(base_question)
        generated_questions = json.dumps(generated_questions, ensure_ascii=False)
        
        # Store in the database
        cursor.execute(
            "UPDATE faq SET questions = ? WHERE id = ?",
            (generated_questions, entry_id)
        )
        conn.commit()
    else:
        continue
# Commit and close

conn.close()

[
    "Aké výhody poskytuje Národná transfúzna služba SR darcom krvi?",
    "Ako je na Slovensku legislatívne upravené dobrovoľné darovanie krvi?",
    "Čo môže poskytovať poskytovateľ zdravotnej starostlivosti darcom krvi podľa zákona?",
    "Aké sú práva zamestnancov, ktorí sa rozhodnú darovať krv počas pracovného času?",
    "Do akej maximálnej doby môže lekár predĺžiť voľno pre zotavenie po darovaní krvi?"
]
[
    "Aké sú zákonné výhody pre nositeľov najmenej striebornej Janského plakety pri poskytovaní ústavnej starostlivosti?",
    "Ako často majú darcovia krvi nárok na preventívnu prehliadku podľa zákona č. 577/2004 Z.z.?",
    "Ktorí poistenci sú podľa zákona oslobodení od úhrady za stravovanie a pobyt na lôžku v liečebných zariadeniach?",
    "Aké sú špeciálne ustanovenia pre preventívne prehliadky poistencov, ktorí nie sú darcami?",
    "V akých prípadoch sa nevzťahujú výhody uvedené v § 38 ods. 9 na poistencov v prírodných liečebných kúpeľoch?"
]
[
    "Ako môžem preukázať o

In [36]:
!pip install langdetect

from langdetect import detect_langs
def detect_language(text):
    # Detect languages and their confidence scores

    # Initialize confidence for English and German

    languages = detect_langs(text)

    # Initialize the confidence for English
    english_confidence = 0.0
    slovak_confidence = 0.0
    # Loop through the detected languages and find the confidence for English
    for lang in languages:
        if lang.lang == "en":
            english_confidence = lang.prob
        elif lang.lang == "sk":
            slovak_confidence = lang.prob
        else:
            break
    return english_confidence, slovak_confidence

res = detect_language("Hello, how are you?")
[print(i) for i in res]
res = detect_language("Ahoj, ako sa máš?")
[print(i) for i in res]

0.8571403618745717
0.0
0.0
0.9999975090968889


[None, None]

In [45]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect("../data/calendar.db")
cursor = conn.cursor()

# Create bookings table
cursor.execute("""
CREATE TABLE IF NOT EXISTS booking (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT NOT NULL,
    timestamp INTEGER NOT NULL,
    donation_type TEXT NOT NULL,
    location TEXT NOT NULL,
    success BOOLEAN NOT NULL,
    status TEXT NOT NULL,
    note TEXT
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS slots (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    location TEXT NOT NULL,
    timestamp INTEGER NOT NULL,
    slots_total INTEGER NOT NULL,
    slots_remaining INTEGER NOT NULL
)
""")


conn.commit()
conn.close()

In [64]:
conn = sqlite3.connect("../data/calendar.db")
import random
from datetime import datetime, timedelta

cursor = conn.cursor()

# Locations
locations = [
    "Banská Bystrica", "Bratislava - Kramáre", "Bratislava - Ružinov", "Košice",
    "Martin", "Nitra", "Nové Zámky", "Poprad", "Prešov", "Trenčín", "Trnava", "Žilina"
]

# Date range: March 11 to March 24, 2025
start_date = datetime(2025, 3, 11, 7, 0)  # 07:00 AM on March 11
end_date = datetime(2025, 3, 24, 15, 0)  # 03:00 PM on March 24

# Generate time slots (every 15 minutes from 07:00 to 15:00)
time_slots_per_day = [(start_date + timedelta(minutes=15 * i)).timestamp() // 60 for i in range(0, 8 * 4)]  # 8 hours × 4 slots per hour

# Insert data for each location and each day
current_date = start_date
while current_date <= end_date:
    for location in locations:
        for timestamp in time_slots_per_day:
            adjusted_timestamp = int(timestamp + (current_date - start_date).total_seconds() // 60)
            slots_total = 3
            slots_remaining = random.randint(0, 3)  # Random available slots

            cursor.execute("""
                INSERT INTO slots (location, timestamp, slots_total, slots_remaining)
                VALUES (?, ?, ?, ?)
            """, (location, adjusted_timestamp, slots_total, slots_remaining))

    current_date += timedelta(days=1)  # Move to the next day

# Commit and close connection
conn.commit()
conn.close()

print("Dummy data inserted successfully!")


Dummy data inserted successfully!


In [18]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect("../data/calendar.db")
cursor = conn.cursor()

# Select sessions where expires is greater than a specific timestamp
cursor.execute("""
DELETE  FROM users WHERE email =  ?
""", ("m.lukan@telekom.com",))

# Fetch and print the results
results = cursor.fetchone()
print(results)


conn.commit()
conn.close()

None


In [55]:
conn = sqlite3.connect("../data/calendar.db")
import random
from datetime import datetime, timedelta

cursor = conn.cursor()

# Locations
locations = [
    "Banská Bystrica", "Bratislava - Kramáre", "Bratislava - Ružinov", "Košice",
    "Martin", "Nitra", "Nové Zámky", "Poprad", "Prešov", "Trenčín", "Trnava", "Žilina"
]

for location in locations:

    cursor.execute("""
        INSERT INTO locations (name)
        VALUES (?)
    """, (location,))


# Commit and close connection
conn.commit()
conn.close()

print("Location data inserted successfully!")

Location data inserted successfully!


In [65]:


def get_unix_minutes(date_obj):
    """Convert a datetime object to Unix timestamp in minutes."""
    return int(date_obj.timestamp() // 60)

def format_timestamp(unix_minutes):
    """
    Converts a Unix timestamp in minutes to a formatted time string.
    
    - If the timestamp is today → returns "HH:MM"
    - If the timestamp is another day → returns "DD.MM. HH:MM"

    :param unix_minutes: Unix timestamp in minutes
    :return: Formatted time string
    """
    dt = datetime.fromtimestamp(unix_minutes * 60)  # Convert to datetime
    today = datetime.now().date()

    if dt.date() == today:
        return dt.strftime("%H:%M")  # Format as "HH:MM"
    else:
        return dt.strftime("%d.%m. %H:%M")  # Format as "DD.MM. HH:MM"


def get_time_slots(db_path, location):
    """
    Retrieves all available time slots between start_time and end_time.

    :param db_path: Path to the SQLite database
    :param start_time: Start time in Unix minutes
    :param end_time: End time in Unix minutes
    :return: List of available timestamps
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT location, timestamp FROM slots 
        WHERE slots_remaining > 0 AND  location = ?
        ORDER BY timestamp ASC
    """, (location,))

    results = [row[1] for row in cursor.fetchall()]
    conn.close()
    return results
res = get_time_slots("../data/calendar.db","Nitra")
for i in res:
    print(format_timestamp(i))


07:00
07:15
07:30
07:45
08:00
08:15
08:30
09:15
09:30
09:45
10:15
10:45
11:00
11:15
11:30
11:45
12:00
12:15
13:00
13:15
13:30
14:00
14:15
14:30
14:45
12.03. 07:00
12.03. 07:15
12.03. 07:30
12.03. 08:00
12.03. 08:15
12.03. 08:30
12.03. 08:45
12.03. 09:00
12.03. 09:15
12.03. 09:30
12.03. 09:45
12.03. 10:00
12.03. 10:15
12.03. 10:45
12.03. 11:00
12.03. 11:15
12.03. 11:30
12.03. 11:45
12.03. 12:15
12.03. 12:30
12.03. 12:45
12.03. 13:00
12.03. 13:15
12.03. 13:30
12.03. 14:00
12.03. 14:15
12.03. 14:30
12.03. 14:45
13.03. 07:00
13.03. 07:15
13.03. 07:45
13.03. 08:00
13.03. 08:15
13.03. 08:30
13.03. 08:45
13.03. 09:15
13.03. 09:45
13.03. 10:00
13.03. 10:15
13.03. 10:30
13.03. 10:45
13.03. 11:00
13.03. 11:30
13.03. 11:45
13.03. 12:00
13.03. 12:15
13.03. 12:45
13.03. 13:00
13.03. 13:45
13.03. 14:15
13.03. 14:45
14.03. 07:00
14.03. 07:30
14.03. 07:45
14.03. 08:00
14.03. 08:30
14.03. 08:45
14.03. 09:15
14.03. 09:30
14.03. 10:00
14.03. 10:15
14.03. 10:30
14.03. 11:15
14.03. 11:30
14.03. 11:45
14.03

In [71]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect("../data/calendar.db")
cursor = conn.cursor()

# Create bookings table
cursor.execute("""
UPDATE users set sex = 'M' where email = 'mlukan@gmail.com' ;
""")

conn.commit()
conn.close()

In [ ]:
import smtplib
import random
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email_verification(email, code, language="en"):
    sender_email = "mlukan@gmail.com"
    app_password = "dkmqmzxhtcaeheqz"
    subject = "Your 2FA Verification Code from the Blood Donation Bot" if language == "en" else "Váš overovací kód od Blood Donation Bota"
    
    # Email content
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = email
    message["Subject"] = subject
    body = f"Your verification code is: \n  <b>{code}</b>" if language == "en" else f"Váš overovací kód je: \n <b>{code}</b>"
    message.attach(MIMEText(body, "plain"))

    try:
        # Connect to Gmail SMTP server
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, email, message.as_string())
        server.quit()
        print("Verification email sent!")
    except Exception as e:
        print(f"Error sending email: {e}")

# Generate a random 6-digit code
verification_code = str(random.randint(100000, 999999))
send_email_verification("mlukan@gmail.com", verification_code)


Verification email sent!


In [8]:
from ruamel.yaml import YAML
yaml=YAML()
import json
import io
def yaml_dump(data):
    dump = io.StringIO()
    # yaml.round_trip_dump(data, dump)
    yaml.dump(data, dump)
    return dump.getvalue()
with open("/Users/A19893678/Documents/Home/Projekty/RasaChallenge2025/data/donation_questionnaire_responses.json", "r", encoding="utf-8") as file:
    data = json.load(file)
with open("../data/slovak_questionnaire_responses.json") as f:
    slovak_data = json.load(f)

responses_out = {}
for name, response in data.items():
    slovak_text = slovak_data[name]
    responses_out[name] = response
    slot = name.split("utter_ask_")[1]
    responses_out[name].append(dict(text=slovak_text,condition=[dict(type="slot", name="slot_lang", value="sk")],
                                    buttons=[
                                        dict(title="Áno",
                                        payload=f"/SetSlots({slot}=yes)",
                                        ),
                                        dict(title="Nie",
                                            payload=f"/SetSlots({slot}=no)",
                                        )]))
print(yaml_dump(dict(responses=responses_out)))

responses:
  utter_ask_slot_q1:
  - text: Have you previously donated blood, plasma, or blood cells?
    buttons:
    - title: Yes
      payload: /SetSlots(slot_q1=yes)
    - title: No
      payload: /SetSlots(slot_q1=no)
  - text: Darovali ste v minulosti krv, plazmu alebo krvné bunky?
    condition:
    - type: slot
      name: slot_lang
      value: sk
    buttons:
    - title: Áno
      payload: /SetSlots(slot_q1=yes)
    - title: Nie
      payload: /SetSlots(slot_q1=no)
  utter_ask_slot_q2:
  - text: Have you ever been permanently excluded from blood donation?
    buttons:
    - title: Yes
      payload: /SetSlots(slot_q2=yes)
    - title: No
      payload: /SetSlots(slot_q2=no)
  - text: Boli ste niekedy v minulosti vyradeny z darovania krvi?
    condition:
    - type: slot
      name: slot_lang
      value: sk
    buttons:
    - title: Áno
      payload: /SetSlots(slot_q2=yes)
    - title: Nie
      payload: /SetSlots(slot_q2=no)
  utter_ask_slot_q3:
  - text: Do you feel health

In [9]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect("../data/calendar.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS forms (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT not null,
    data TEXT,
    timestamp TEXT,
    expires TEXT,
    form_id TEXT
)
""")
conn.commit()
conn.close()

In [22]:
from datetime import datetime
ts=28941630
def format_timestamp(unix_minutes):
    """
    Converts a Unix timestamp in minutes to a formatted time string.
    
    - If the timestamp is today → returns "HH:MM"
    - If the timestamp is another day → returns "DD.MM. HH:MM"

    :param unix_minutes: Unix timestamp in minutes
    :return: Formatted time string
    """
    dt = datetime.fromtimestamp(unix_minutes * 60)  # Convert to datetime
    today = datetime.now().date()

    if dt.date() == today:
        return dt.strftime("%H:%M")  # Format as "HH:MM"
    else:
        return dt.strftime("%d.%m.%y %H:%M")  # Format as "DD.MM. HH:MM"
print(format_timestamp(ts))

10.01.25 09:30
